# 基于一等函数的设计模式实现

## 语言与设计模式

设计模式与语言无关，但是这并不意味着每一个模式都需要依照相同的方式在不同语言中实现。考虑到语言的特性各不相同，很可能在一种语言中需要单独设计并实现的设计模式，另一种语言却直接具有相应的特性。

《设计模式：可复用面向对象软件的基础》的作者在引言中有如下描述：
程序设计的语言非常重要，它将影响人们理解问题的出发点......若我们采用过程式语言，可能就要包括诸如“继承”、“封装”和“多态”的设计模式。相应的，一些特殊的面向对象语言可以直接支持我们的某些模式，例如CLOS支持多方法概念，这就减少了访问者模式的必要性。

同样的，Python作为具有一等函数概念的语言，某些设计模式的python实现不同于其在C++中的实现，例如“策略”、“命令”以及“访问者”等模式。

值得注意的是，上述列举的设计模式均是行为模式，实际上一等函数的引入改变了一些“对象”的实现方法，对于Python来说：Everything in Python is an object。Python的这一特性使得一些行为模式的实现得以简化。

## “策略”模式的Python实现

策略模式：定义一系列算法，把他们一个个封装起来，并且使它们可相互替换。本模式使得算法可独立于使用它的客户而变化。

策略模式描述了在一个需要根据上下文内容选择合适算法的问题中，如何更好的组织这些算法的设计模式。

策略模式有如下关键组件：

* 上下文：提供进行算法选择的关键信息
* 策略：不同算法的共用接口
* 具体策略：策略的子类，不同算法的实现

策略模式若依照C++中的实现方法，需要分别实现用于描述上下文的Context类、用于描述策略共用接口的Strategy类以及不同策略的具体实现ConcreteStrategy类。值得注意的是，策略模式并没有包含策略选择的方法，在一般的策略模式中，使用哪一个策略要求用户决定。

下述按照上述的步骤实现了一个计算购物折扣的策略模式，其中，Order管理上下文，LineItem管理上下文中的购物车信息，Customer管理上下文中的用户信息，Promotion建立计算折扣的同一基类，FidelityPromo、BulkItemPromo和LargeOrderPromo则分别实现了三种具体策略。

可以注意到，FidelityPromo、BulkItemPromo和LargeOrderPromo均是仅实现了一个方法的类并且不需要维护内部状态。Python中函数是一等公民，函数可以作为参数传递给其他函数，因此上述的三个具体策略对应的类完全可以替换为函数。

此外，用类实现的具体策略会在每一个上下文中创建实例，为了避免不必要的资源消耗，策略模式通常需要结合使用“享元”，以实现在多个上下文中共享这些具体策略。另一方面，函数更轻量化，并且天生是可共享对象——可以在多个上下文中使用。

## 总结

* 不同语言有不同的特性，同一个设计模式在不同语言中有不同的实现方法，甚至是否需要去实现某一特定的设计模式都与使用的语言类型息息相关
* Python中的函数为一等公民。这使得对于“策略”或者“命令”这样的设计模式，完全可以使用函数代替那些仅有一个方法并且不需要维护内部状态的类。使用函数替代类不仅能实现类似的功能并且能够直接回避在不同上下文中创建类实例带来的资源消耗问题。 

In [ ]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple("Customer", "name fidelity")


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity


class Order:  # 上下文
    
    def __init__(self, customer, cart, promotion=None):

        self.customer = customer
        self.cart = cart
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, "__total"):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):

        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = "<Order total: {:.2f} due: {:.2f}>"
        return fmt.format(self.total(), self.due())


class Promotion(ABC):  # 策略

    @abstractmethod
    def discount(self, order):
        """返回折扣"""


class FidelityPromo(Promotion):  # 第一种具体策略

    def discount(self, order):
        return order.total() * 0.5 if order.customer.fidelity >= 1000 else 0


class BulkItemPromo(Promotion):  # 第二种具体策略

    def discount(self, order):
        _discount = 0

        for item in order.cart:
            if item.quantity >= 20:
                _discount += item.total() * 0.1

        return _discount


class LargeOrderPromo(Promotion):  # 第三种具体策略

    def discount(self, order):
        discount_items = {item.product for item in order.cart}

        if len(discount_items) >= 10:
            return order.total() * 0.07
        return 0


def fidelity_promo(order):
    return order.total() * 0.5 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    _discount = 0

    for item in order.cart:
        if item.quantity >= 20:
            _discount += item.total() * 0.1

    return _discount

def large_order_promo(order):
    discount_items = {item.product for item in order.cart}

    if len(discount_items) >= 10:
        return order.total() * 0.07
    return 0
